
# RNN Tutorial for DNA Sequence Classification

In this tutorial, we'll build a Recurrent Neural Network (RNN) to classify DNA sequences using PyTorch. This is a common bioinformatics task that's well-suited for sequence models like RNNs.

## Table of Contents
1. [Introduction to RNNs](#introduction-to-rnns)
2. [Setting Up the Environment](#setting-up-the-environment)
3. [Preparing DNA Sequence Data](#preparing-dna-sequence-data)
4. [Building the RNN Model](#building-the-rnn-model)
5. [Training the Model](#training-the-model)
6. [Evaluating and Testing](#evaluating-and-testing)
7. [Advanced Techniques](#advanced-techniques)
8. [Conclusion](#conclusion)

## Introduction to RNNs

Recurrent Neural Networks (RNNs) are a class of neural networks designed to work with sequential data. Unlike feedforward networks, RNNs have connections that form cycles, allowing information to persist across processing steps. This makes them particularly useful for tasks like:

- Natural language processing
- Time series prediction
- **Biological sequence analysis (our focus today)**

RNNs process sequences one element at a time while maintaining an internal "memory" of what they've seen before. This is perfect for DNA sequences, which are naturally sequential data.


In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt

## Preparing DNA Sequence Data

### Step 1: Load and explore your dataset

Let's assume you have a dataset of DNA sequences and their corresponding labels. We'll create a PyTorch dataset class to handle this data:


In [ ]:
class DNASequenceDataset(Dataset):
    def __init__(self, sequences, labels):
        self.sequences = sequences
        self.labels = labels

        # Define nucleotide mapping
        self.nucleotide_to_idx = {'A': 0, 'C': 1, 'G': 2, 'T': 3}

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        sequence = self.sequences[idx]
        label = self.labels[idx]

        # Convert sequence to numerical representation
        sequence_encoded = [self.nucleotide_to_idx[nucleotide] for nucleotide in sequence]

        # Convert to PyTorch tensors
        sequence_tensor = torch.tensor(sequence_encoded, dtype=torch.long)
        label_tensor = torch.tensor(label, dtype=torch.long)

        return sequence_tensor, label_tensor

### Step 2: Load your dataset and split into train/validation/test sets


In [ ]:
# Example data loading (replace with your actual data loading code)
def load_dna_data():

    df = pd.read_csv("https://raw.githubusercontent.com/sdgroeve/Machine_Learning_course_UGent_D012554_data/master/practicum/Classification/acceptor_sites_dataset_train.csv")
    sequences = df['sequence'].values
    labels = df['label'].values
    labels = [0 if label == -1 else 1 for label in labels]
    return sequences, labels

# Load data
sequences, labels = load_dna_data()

# Split into train, validation, and test sets
seqs_train, seqs_temp, labels_train, labels_temp = train_test_split(
    sequences, labels, test_size=0.3, random_state=42
)
seqs_val, seqs_test, labels_val, labels_test = train_test_split(
    seqs_temp, labels_temp, test_size=0.5, random_state=42
)

print(f"Training samples: {len(seqs_train)}")
print(f"Validation samples: {len(seqs_val)}")
print(f"Testing samples: {len(seqs_test)}")

### Step 3: Create data loaders

Question: How do we deal with different sequence lengths in the dataset?


In [ ]:
# We need to pad sequences to the same length for batch processing
def collate_fn(batch):
    sequences, labels = zip(*batch)

    # Find the length of the longest sequence
    max_length = max(len(seq) for seq in sequences)

    # Pad sequences to the same length
    padded_sequences = []
    for seq in sequences:
        padded_seq = torch.zeros(max_length, dtype=torch.long)
        padded_seq[:len(seq)] = seq
        padded_sequences.append(padded_seq)

    # Stack sequences and labels into batches
    sequences_batch = torch.stack(padded_sequences)
    labels_batch = torch.tensor(labels, dtype=torch.long)

    return sequences_batch, labels_batch

# Create datasets
train_dataset = DNASequenceDataset(seqs_train, labels_train)
val_dataset = DNASequenceDataset(seqs_val, labels_val)
test_dataset = DNASequenceDataset(seqs_test, labels_test)

# Create data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=batch_size, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=batch_size, collate_fn=collate_fn)

## Building the RNN Model

Now, let's define our RNN model for DNA sequence classification:

In [ ]:
class DNASequenceRNN(nn.Module):
    def __init__(self, input_size, embedding_dim, hidden_dim, output_size, num_layers=1, dropout=0.2):
        super(DNASequenceRNN, self).__init__()

        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        # Embedding layer to convert nucleotide indices to dense vectors
        self.embedding = nn.Embedding(input_size, embedding_dim)

        # RNN layer (can be vanilla RNN, LSTM, or GRU)
        self.rnn = nn.GRU(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0
        )

        # Output layer
        self.fc = nn.Linear(hidden_dim, output_size)

        # Dropout for regularization
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # x shape: (batch_size, sequence_length)

        # Embed nucleotides
        x = self.embedding(x)  # (batch_size, sequence_length, embedding_dim)

        # Initialize hidden state
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)

        # Pass through RNN
        out, _ = self.rnn(x, h0)  # out: (batch_size, sequence_length, hidden_dim)

        # We only need the output from the last time step
        out = out[:, -1, :]  # (batch_size, hidden_dim)

        # Apply dropout for regularization
        out = self.dropout(out)

        # Pass through the fully connected layer
        out = self.fc(out)  # (batch_size, output_size)

        return out

### Step 4: Initialize model hyperparameters

In [ ]:
# Model hyperparameters
input_size = 4  # Number of nucleotides (A, C, G, T)
embedding_dim = 8  # Size of embedding vectors
hidden_dim = 64  # Size of hidden layer
output_size = 2  # Number of classes (binary classification)
num_layers = 2  # Number of RNN layers
dropout = 0.2  # Dropout rate

# Create model
model = DNASequenceRNN(input_size, embedding_dim, hidden_dim, output_size, num_layers, dropout)

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

print(model)

## Training the Model



In [ ]:
# Training function
def train(model, train_loader, val_loader, criterion, optimizer, num_epochs, device):
    # Lists to store metrics
    train_losses = []
    val_losses = []
    val_accuracies = []

    # Training loop
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0.0

        for sequences, labels in train_loader:
            sequences, labels = sequences.to(device), labels.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(sequences)
            loss = criterion(outputs, labels)

            # Backward pass and optimize
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * sequences.size(0)

        # Calculate average training loss
        train_loss = train_loss / len(train_loader.dataset)
        train_losses.append(train_loss)

        # Validation phase
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            for sequences, labels in val_loader:
                sequences, labels = sequences.to(device), labels.to(device)

                # Forward pass
                outputs = model(sequences)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * sequences.size(0)

                # Calculate accuracy
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        # Calculate average validation loss and accuracy
        val_loss = val_loss / len(val_loader.dataset)
        val_losses.append(val_loss)

        val_accuracy = correct / total
        val_accuracies.append(val_accuracy)

        # Print metrics
        print(f'Epoch {epoch+1}/{num_epochs}:')
        print(f'  Train Loss: {train_loss:.4f}')
        print(f'  Val Loss: {val_loss:.4f}')
        print(f'  Val Accuracy: {val_accuracy:.4f}')

    return train_losses, val_losses, val_accuracies

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 50
train_losses, val_losses, val_accuracies = train(model, train_loader, val_loader, criterion, optimizer, num_epochs, device)

# Plot training history
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')

plt.subplot(1, 2, 2)
plt.plot(val_accuracies, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Validation Accuracy')

plt.tight_layout()
plt.show()

## Evaluating and Testing

After training, let's evaluate our model on the test set:

In [ ]:
def evaluate(model, test_loader, criterion, device):
    model.eval()
    test_loss = 0.0
    predictions = []
    true_labels = []

    with torch.no_grad():
        for sequences, labels in test_loader:
            sequences, labels = sequences.to(device), labels.to(device)

            # Forward pass
            outputs = model(sequences)
            loss = criterion(outputs, labels)

            test_loss += loss.item() * sequences.size(0)

            # Get predictions
            _, predicted = torch.max(outputs, 1)

            # Store predictions and true labels
            predictions.extend(predicted.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    # Calculate average test loss
    test_loss = test_loss / len(test_loader.dataset)

    # Calculate metrics
    accuracy = accuracy_score(true_labels, predictions)
    report = classification_report(true_labels, predictions)

    return test_loss, accuracy, report

# Evaluate on test set
test_loss, test_accuracy, test_report = evaluate(model, test_loader, criterion, device)

print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")
print("Classification Report:")
print(test_report)


### Step 5: Making predictions with the trained model

In [ ]:
def predict_sequence(model, sequence, device):
    model.eval()

    # Convert sequence to indices
    nucleotide_to_idx = {'A': 0, 'C': 1, 'G': 2, 'T': 3}
    sequence_encoded = [nucleotide_to_idx[nucleotide] for nucleotide in sequence]
    sequence_tensor = torch.tensor(sequence_encoded, dtype=torch.long).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(sequence_tensor)
        _, predicted = torch.max(output, 1)

        # Get probability scores
        probabilities = torch.nn.functional.softmax(output, dim=1)[0]

    return predicted.item(), probabilities.cpu().numpy()

# Example usage
example_sequence = "ATCGATCGATCGATCG"
predicted_class, probabilities = predict_sequence(model, example_sequence, device)

print(f"Sequence: {example_sequence}")
print(f"Predicted class: {predicted_class}")
print(f"Class probabilities: {probabilities}")

## Bonus: Using Attention mechanism 

In [ ]:
class AttentionDNASequenceRNN(nn.Module):
    def __init__(self, input_size, embedding_dim, hidden_dim, output_size, num_layers=1, dropout=0.2):
        super(AttentionDNASequenceRNN, self).__init__()

        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        # Embedding layer
        self.embedding = nn.Embedding(input_size, embedding_dim)

        # GRU
        self.rnn = nn.GRU(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0
        )

        # Attention layer
        self.attention = nn.Linear(hidden_dim, 1)

        # Output layer
        self.fc = nn.Linear(hidden_dim, output_size)

        # Dropout
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # Embed nucleotides
        x = self.embedding(x)

        # Initialize hidden state
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)

        # Pass through RNN
        out, _ = self.rnn(x, h0)  # out: (batch_size, seq_len, hidden_dim)

        # Calculate attention weights
        attention_scores = self.attention(out).squeeze(-1)  # (batch_size, seq_len)
        attention_weights = torch.nn.functional.softmax(attention_scores, dim=1)

        # Apply attention weights
        attention_weights = attention_weights.unsqueeze(-1)  # (batch_size, seq_len, 1)
        weighted_output = out * attention_weights  # (batch_size, seq_len, hidden_dim)
        context_vector = weighted_output.sum(dim=1)  # (batch_size, hidden_dim)

        # Apply dropout
        context_vector = self.dropout(context_vector)

        # Pass through the fully connected layer
        output = self.fc(context_vector)

        return output

